In [5]:
import numpy as np
from pathlib import Path

DATA_PROC = Path("/Users/apple/Documents/Computational Neuroscience Paris Saclay/"
    "Supervised projects/main/code/data/processed")
DATA_PROC.mkdir(parents=True, exist_ok=True)

# CHANGE ONLY THESE TWO LINES:
PARC_NAME = "Sch200"  # "Sch200"
N_NODES   = 200       # 200

print("Parcellation:", PARC_NAME)
print("Nodes:", N_NODES)



Parcellation: Sch200
Nodes: 200


In [6]:
# ===== Synthetic SC (temporary until you get real Schaefer matrices) =====
def make_synthetic_sc(n, sigma=10.0):
    i = np.arange(n).reshape(-1,1)
    j = np.arange(n).reshape(1,-1)
    dist = np.abs(i - j)
    W = np.exp(-(dist**2) / (2 * sigma**2))
    np.fill_diagonal(W, 0)
    return W

# ===== Laplacian =====
def sym_norm_laplacian(W, eps=1e-10):
    d = W.sum(axis=1)
    d = np.maximum(d, eps)
    D_inv_sqrt = np.diag(1.0 / np.sqrt(d))
    return np.eye(W.shape[0]) - D_inv_sqrt @ W @ D_inv_sqrt

# ===== Diffusion =====
def simulate_diffusion(W, T=1000, alpha=0.6, noise_std=0.1, seed=None):
    rng = np.random.default_rng(seed)
    N = W.shape[0]
    L = sym_norm_laplacian(W)
    lam_max = np.linalg.eigvalsh(L).max()
    A = np.eye(N) - alpha * L / lam_max
    X = np.zeros((N, T))
    X[:,0] = rng.normal(0, 1, size=N)
    for t in range(T-1):
        noise = rng.normal(0, noise_std, size=N)
        X[:,t+1] = A @ X[:,t] + noise
    return X

# ===== Hopf =====
def simulate_hopf(W, T=2000, dt=0.02, a=0.3, w0=0.05, coupling=0.2, noise_std=0.02, seed=None):
    rng = np.random.default_rng(seed)
    N = W.shape[0]
    z = rng.normal(0,0.1,size=N) + 1j*rng.normal(0,0.1,size=N)
    X = np.zeros((N,T))
    row_sum = W.sum(axis=1)

    for t in range(T):
        X[:,t] = z.real
        coupling_term = W @ z - row_sum * z
        dz = (a + 1j*w0)*z - (np.abs(z)**2)*z + coupling * coupling_term
        z = z + dt * dz
        noise_real = rng.normal(0, noise_std, size=N)
        noise_imag = rng.normal(0, noise_std, size=N)
        z = z + np.sqrt(dt)*(noise_real + 1j*noise_imag)
    return X


In [7]:
# === 1. Structural Connectome ===
W = make_synthetic_sc(N_NODES, sigma=10.0)
np.save(DATA_PROC / f"W_syn_{PARC_NAME}.npy", W)
print("Saved:", f"W_syn_{PARC_NAME}.npy")

# === 2. Diffusion: noise = 0.0 ===
Y_clean = simulate_diffusion(W, T=1000, noise_std=0.0, seed=0)
np.save(DATA_PROC / f"Y_diff_{PARC_NAME}_noise0_mean.npy", Y_clean)
print("Saved diffusion (noise=0):", f"Y_diff_{PARC_NAME}_noise0_mean.npy")

# === 3. Diffusion: noise = 0.1 averaged over 10 seeds ===
runs = []
for seed in range(10):
    X = simulate_diffusion(W, T=1000, noise_std=0.1, seed=seed)
    runs.append(X)
Y_high = np.mean(runs, axis=0)
np.save(DATA_PROC / f"Y_diff_{PARC_NAME}_noise01_mean.npy", Y_high)
print("Saved diffusion (noise=0.1):", f"Y_diff_{PARC_NAME}_noise01_mean.npy")


Saved: W_syn_Sch200.npy
Saved diffusion (noise=0): Y_diff_Sch200_noise0_mean.npy
Saved diffusion (noise=0.1): Y_diff_Sch200_noise01_mean.npy


In [8]:
Y_hopf = simulate_hopf(W, T=2000, dt=0.01, a=0.1, w0=0.05,
                       coupling=0.1, noise_std=0.01, seed=0)
np.save(DATA_PROC / f"Y_hopf_synthetic_{PARC_NAME}.npy", Y_hopf)
print("Saved Hopf:", f"Y_hopf_synthetic_{PARC_NAME}.npy")

print("\n=== ALL DONE ===")


Saved Hopf: Y_hopf_synthetic_Sch200.npy

=== ALL DONE ===
